In [ ]:
# default_exp mail

In [ ]:
#export
from dataclasses import dataclass
from typing import Optional
import base64
import os
import os.path
import pathlib
import pickle

from email.mime.application import MIMEApplication
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import fastcore.test
import mimetypes

import send_to_pb.utils as u


In [ ]:
#export

# If modifying these scopes, delete the file token.pickle.
SCOPES = [
            'https://www.googleapis.com/auth/gmail.send',
            'https://www.googleapis.com/auth/gmail.readonly',
         ]


@dataclass
class MessageEncoded:
    raw: str

    def dict(self):
        return {"raw": self.raw}

In [ ]:
#export

class Emailer:
    def __init__(self):
        self.service = self.g_auth()

    def g_auth(self):
        """Shows basic usage of the Gmail API.
        Lists the user's Gmail labels.
        """
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    pathlib.Path('~/credentials/gmail_api_creds.json').expanduser(),
                    SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        service = build('gmail', 'v1', credentials=creds)
        return service


    def create_message_with_attachment(self,
                                       file:pathlib.Path,
                                       message_text:str = '',
                                       sender:str="David",
                                       to:str="dzlob@pbsync.com",
                                       subject:str="SEND TO PBSYNC",
                                       ) -> MessageEncoded:
        
        if not file.is_file():
            raise FileNotFoundError(file)

        message = MIMEMultipart()
        message['to'] = to
        message['from'] = sender
        message['subject'] = subject

        if message_text == '':
            message_text = "# THIS MESSAGE WAS AUTOGENERATED BY SEND_TO_PB"
        message.attach(
            MIMEText(message_text)
            )

        content_type, encoding = mimetypes.guess_type(file)

        if content_type is None or encoding is not None:
            content_type = 'application/octet-stream'

        main_type, sub_type = content_type.split('/', 1)

        if main_type == 'text':
            fp = open(file, 'rb')
            msg = MIMEText(fp.read().decode("utf-8"), _subtype=sub_type)
            fp.close()

        elif main_type == 'application' and sub_type == 'pdf':
            temp = open(file, 'rb')
            msg = MIMEApplication(temp.read(), _subtype=sub_type)
            temp.close()
        elif main_type == 'image':
            fp = open(file, 'rb')
            msg = MIMEImage(fp.read(), _subtype=sub_type)
            fp.close()
        elif main_type == 'audio':
            fp = open(file, 'rb')
            msg = MIMEAudio(fp.read(), _subtype=sub_type)
            fp.close()
        else:
            fp = open(file, 'rb')
            msg = MIMEBase(main_type, sub_type)
            msg.set_payload(fp.read())
            fp.close()
        filename = os.path.basename(file)
        msg.add_header('Content-Disposition', 'attachment', filename=filename)
        message.attach(msg)

        ## Part 4 encode the message (the message should be in bytes)
        message_as_bytes = message.as_bytes() # the message should converted from string to bytes.
        message_as_base64 = base64.urlsafe_b64encode(message_as_bytes) #encode in base64 (printable letters coding)

        raw = message_as_base64.decode()  # need to JSON serializable (no idea what does it means)
        return MessageEncoded(raw)


    def send_message(self, content:MessageEncoded, user_id:Optional[str]=''):
        if user_id == '':
            user_id = 'david.dobrinskiy@gmail.com'

        try:
            body = {"raw": content.raw}
            gmail_message = (self.service
                             .users()
                             .messages()
                             .send(userId=user_id, body=body)
                             )
            gmail_message.execute()

            u.logger.info(f"Gmail message sent as {user_id}")
            return gmail_message
        except Exception as e:
            u.logger.error(f'An error occurred: {e}')
            return None

In [ ]:
#hide

def create_message_without_attachment(sender, to, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    raw_message = base64.urlsafe_b64encode(message.as_string().encode("utf-8"))
    return {
        'raw': raw_message.decode("utf-8")
    }


def create_draft(service, user_id, message_body):
    try:
        message = {'message': message_body}
        draft = service.users().drafts().create(userId=user_id, body=message).execute()
        print("Draft id: %s\nDraft message: %s" % (draft['id'], draft['message']))

        return draft
    except Exception as e:
        print('An error occurred: %s' % e)
        return None

In [ ]:
# integration
dir = pathlib.Path("~/Downloads/send_to_pb/").expanduser()
file = next(dir.glob('*.pdf'))

emailer = Emailer()
content = emailer.create_message_with_attachment(file=file,
                                                 to="david.dobrinskiy@gmail.com",
                                                 )
emailer.send_message(content=content)

2020-10-16 14:13:33.774 | INFO     | __main__:send_message:111 - Gmail message sent as david.dobrinskiy@gmail.com


In [ ]:
# integration
def send_non_existent_file():
    emailer = Emailer()
    content = emailer.create_message_with_attachment(file=pathlib.Path("does not exist.txt"))

fastcore.test.test_fail(send_non_existent_file, "Should err on non-existent file")